# Introduction to PyCBC Inference 2: Analyzing a gravitational wave
### Collin Capano

In this tutorial we show how to set up a run for a gravitation wave, in this case, GW150914.

### Prerequisites

We will need the most recent version of pycbc installed for this tutorial.

In [1]:
!pip install lalsuite pycbc

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
from IPython.display import Image
from IPython.core.display import HTML

## BBH example

We will use the provided example configuration file [bbh_example.ini](bbh_example.ini). This is the configuration file you would need to analyze signals like GW150914.

In the following sections, we look at the various sections of the configuration file in detail.

## The model

The model we will use is set by the `[model]` section. By setting the `name = gaussian_noise`, we tell `pycbc_inference` that we want to use the [GaussianNoise](https://pycbc.org/pycbc/latest/html/pycbc.inference.models.html#pycbc.inference.models.gaussian_noise.GaussianNoise) model. This means that we will be analyzing data $d$ from one or more detectors, which is sampled at some constant sample rate $1/\Delta t$ for a period of time $T$, yielding $N = T/\Delta t$ samples (more on how to load data below).

This model assumes that the data consists of stationary Gaussian noise plus a signal $h$. The signal is modeled by a waveform model, which depends on several parameters. Which model to use is determined by the `approximant` parameter, which is set in the `static_params` section (see below).

When this model is given a set of parameter values $\vec{\vartheta}$, it generates a frequency-domain waveform $\tilde{h}(\vec{\vartheta})$ using the waveform model. It then calculates the log likelihood:
\begin{equation}
\log p(d|\vec{\vartheta}, h) =  -\frac{1}{2} \sum_i \left< h_i(\vec{\vartheta}) - d_i,\, h_i(\vec{\vartheta}) - d_i \right>,
\end{equation}
where the sum is over the number of detectors. The inner product is given by:
\begin{equation}
\left<a_i, \, b_i\right> = 4 \Re \sum_{k=k_0}^{N/2} \frac{\tilde{a}_i^{*}(k \Delta f) \tilde{b}_i(k \Delta f)}{S^{(i)}_n(k\Delta f)} \Delta f.
\end{equation}
Here, $S_n^{(i)}$ is the power spectral density of the noise in the $i$th detector and $\Delta f = 1/T$ is the frequency resolution. This is the discrete form of the matched filter (see Matt Pitkin's talk).

As we see, the model requires a lower frequency cutoff for the inner product $f_0 = k_0 \Delta f$. This is set for each detector by setting the `{detector}-low-frequency-cutoff` option where `{detector}` is the detector name.

## The variable and static params

As with the Normal 2D example, we see that there is a `[variable_params]` section. This lists the names of all of the parameters we will be varying in the run. Now we see that there is also a `[static_params]` section. These are parameters that will be kept fixed throughout the run. In this example we have:
 * `approximant = IMRPhenomPv2`: this means that the waveform model we use will be IMRPhenomPv2
 * `f_lower = 20`: this sets the starting frequency for the waveform generation to 20Hz. *This is separate from the low frequency cutoff of the inner product, which is set in the `[model]` section.*
 * `f_ref = 20`: the "reference" frequency of the waveform.

The `approximant` argument determines what type of waveform we will be generating. Since it is set to `IMRPhenomPv2`, we will be generating a CBC waveform.

## Setting the prior

**Every parameter listed in the `[variable_params]` section must have a prior specified in the file.** This is done by adding sections named `[prior-{params}]` to the file, where `{param}` is the name of the parameter that the section sets a prior for; e.g., `[prior-mass1]`. A section may provide a distribution for multiple parameters. In that case, all of the parameters must be listed in the header as a `+` separated list. For example, `[prior-ra+dec]`. The order that the parameters are provided does not matter.

Each prior section must also have a `name` set. This specifies the name of the distribution to use for that parameter. Distributions are defined in PyCBC's [distributions package](https://pycbc.org/pycbc/latest/html/pycbc.distributions.html); several are available for use. For the complete list, see the table [here](https://pycbc.org/pycbc/latest/html/inference.html#configuring-the-prior).

The rest of the settings in the `[prior]` sections depend on the distribution being used. For example, the `uniform` distribution requires minimum and maximum bounds to be provided for each parameter:
```
[prior-mass1]
name = uniform
min-mass1 = 10.
max-mass1 = 80.
```
Some distributions require no options, since they are predefined in the code. For example, the `uniform_sky` distribution provides the appropriate distributions for right ascension and declination, which it expects to be called `ra` and `dec`, respectively. This is why that section is simply:
```
[prior-ra+dec]
name = uniform_sky
```

### Challenge:
 * The configuration file is missing a prior for the coalescene time `tc`. We know that the coalescence time of GW150914 is approximately 1126259462.42 (GPS seconds). Set a uniform prior on `tc` equal to this time $\pm$ 0.1 s.

## The sampler

Here we are using the `emcee_pt` sampler. This is a parallel tempered sampler, so it requires a number of temperatures to be set. Note a few other differences from the analytic example here:
  * The `burn-in-test` is set to `nacl & max_posterior`.
  * Instead of an `niterations` option, we have `effective-nsamples = 1000`.
  * A checkpoint interval is set: `checkpoint-interval = 2000`
  * There is a `max-samples-per-chain` option.

These are all options specific to MCMC samplers like `emcee_pt`. Details:

#### The burn in option
Multiple burn in tests may be combined using standard boolean operators like `&` and `|`. In this example, we will consider the sampler to be burned in when it has passed the `nacl` test *and* the `max_posterior` test. This means:

 * `nacl`: The second half of the chain must be longer than 5 times the ACL. If so, the samlper is considered burned-in at the halfway point.
 * `max_posterior`: All of the walkers must find a point that has a log posterior greater than `maxP - ndim/2`, where `maxP` is the maximum posterior value found over all the walkers and `ndim` is the number of variable parameters. The first iteration for which all the walkers pass this test is the burn in iteration.
 
By doing `&`, we take the larger iteration of these two tests. This combination of tests has worked well for `emcee_pt`.

#### Checkpointing
When a `checkpoint-interval` is set, `pycbc_inference` will dump the results to a checkpoint file after every `checkpoint-interval` iterations. The checkpoint file has the same name as the output, but with `.checkpoint` added on to it. 

While ``pycbc_inference`` is running it will create a checkpoint file which
is named ``{output-file}.checkpoint``, where ``{output-file}`` was the name
of the file you specified with the ``--output-file`` command. If a `checkpoint-interval` is set, `pycbc_inference` will checkpoint after the given number of iterations. For `emcee_pt`, this means that it will dump the current samples to this file; when finished, the file is
renamed to ``{output-file}``.

A ``{output-file}.bkup`` is also created, which is a copy of the checkpoint file. This is kept in case the checkpoint file gets corrupted during writing. The ``.bkup`` file is deleted at the end of the run, unless ``--save-backup`` is turned on.

If `pycbc_inference` is terminated while running (either by error, or by a system interrupt), the checkpoint and bkup files remain. When `pycbc_inference` is restarted, it will check for those files. If they are found, it will resume from where it last left off.

#### Termination condition
By setting `effective-nsamples` we tell the `pycbc_inference` until it has an effective number of samples greater than or equal to the specified value. Effective samples are the number of independent samples of the posterior we have. This is given by number of samples that remain after burn-in and thinned by the autocorelation time.

The number of effective samples are counted at each checkpoint. For this reason, a checkpoint-interval must be provided if `effective-nsamples` is set.

#### Max samples per chain
If `max-samples-per-chain` is provided, `pycbc_inference` will ensure that no more than the given number of samples per chain are stored in the output file. Samples will be thinned on disk and in memory when a checkpoint happens to ensure this. This is important for keeping file size down. Without it, a GW run with `1000` walkers and `4` temps can result in a file that is over 100GB, since every sample will be saved. With `max-samples-per-chain = 1000`, the maximum file size is capped to ~1GB.

### Samping and waveform transforms
You'll note a `waveform_transforms` and `sampling_transforms` sections. Those are described in more detail in the next tutorial.

## Data settings

Options for loading the gravitational wave data are set on the command line. See [run_bbh_example.sh](run_bbh_example.sh) for details.

### Challenge:
 * The data that will be analyzed is set by the `--gps-start-time` and `--gps-end-time` options. What will these be set to by the script? How much time is analyzed?
 * The longest waveform admitted by the prior is ~6s long. So why is the analyzed time longer than 6 seconds? (Hint: Recall FFT wrap around.)
 
*Note: in a future release, data options will be moved from the command line and into the config file.*

## Run it

Before we can run, we need to download frame files from [GWOSC](https://www.gw-openscience.org/about/). These contain the LIGO data that we will analyze.

In [8]:
if not os.path.exists('H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf
if not os.path.exists('L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf

--2019-05-12 10:40:33--  https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf
Resolving www.gw-openscience.org (www.gw-openscience.org)... 131.215.125.179
Connecting to www.gw-openscience.org (www.gw-openscience.org)|131.215.125.179|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf/ [following]
--2019-05-12 10:40:35--  https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf/
Reusing existing connection to www.gw-openscience.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 130069555 (124M) [application/x-gwf]
Saving to: ‘H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf’

H-H1_GWOSC_4KHZ_R1- 100%[===================>] 124.04M   118KB/s    in 18m 57s 

2019-05-12 10:58:51 (112 KB/s) - ‘H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf’ saved [130069555/130069555]

--201

To run this example, we will just run the bash script. Running this example to completion will take several hours. Instead, we'll just run for a couple of checkpoints, kill it, then start it again to see how checkpointing works.

First, do the following:
 * Set the checkpoint interval in the config file to 4.

Now run the script. Watch the output. After it checkpoints (it will say "Running sampler for 4 to 8 iterations"), interrupt the kernel by hitting the stop button above.

In [10]:
!bash run.sh

2019-05-12 14:07:00,297 Using seed 39392
2019-05-12 14:07:00,299 Running with CPU support: 1 threads
2019-05-12 14:07:00,956 Reading Frames
2019-05-12 14:07:11,037 Highpass Filtering
2019-05-12 14:07:11,050 Converting to float64
2019-05-12 14:07:11,050 Resampling data
2019-05-12 14:07:11,083 Highpass Filtering
2019-05-12 14:07:11,087 Remove Padding
2019-05-12 14:07:11,088 Reading Frames
2019-05-12 14:07:20,871 Highpass Filtering
2019-05-12 14:07:20,877 Converting to float64
2019-05-12 14:07:20,877 Resampling data
2019-05-12 14:07:20,890 Highpass Filtering
2019-05-12 14:07:20,893 Remove Padding
2019-05-12 14:07:20,893 Will generate a different time series for PSD estimation
2019-05-12 14:07:20,894 Reading Frames
2019-05-12 14:07:29,952 Highpass Filtering
2019-05-12 14:07:30,174 Converting to float64
2019-05-12 14:07:30,186 Resampling data
2019-05-12 14:07:31,039 Highpass Filtering
2019-05-12 14:07:31,161 Remove Padding
2019-05-12 14:07:31,162 Reading Frames
2019-05-12 14:07:40,453 Highp

Now run `ls` to see the files in the diretory. You should see a `bbh_results.hdf.checkpoint` and `bbh_results.hdf.bkup`. These are your checkpoint and backup files:

In [5]:
!ls bbh_results*

bbh_results.hdf.bkup       bbh_results.hdf.checkpoint


Run the script again. Watch the messages. You should see it say that it is starting from iteration 4. Stop it after it has gotten to another checkpoint.

In [10]:
!bash run.sh

2019-05-12 14:07:00,297 Using seed 39392
2019-05-12 14:07:00,299 Running with CPU support: 1 threads
2019-05-12 14:07:00,956 Reading Frames
2019-05-12 14:07:11,037 Highpass Filtering
2019-05-12 14:07:11,050 Converting to float64
2019-05-12 14:07:11,050 Resampling data
2019-05-12 14:07:11,083 Highpass Filtering
2019-05-12 14:07:11,087 Remove Padding
2019-05-12 14:07:11,088 Reading Frames
2019-05-12 14:07:20,871 Highpass Filtering
2019-05-12 14:07:20,877 Converting to float64
2019-05-12 14:07:20,877 Resampling data
2019-05-12 14:07:20,890 Highpass Filtering
2019-05-12 14:07:20,893 Remove Padding
2019-05-12 14:07:20,893 Will generate a different time series for PSD estimation
2019-05-12 14:07:20,894 Reading Frames
2019-05-12 14:07:29,952 Highpass Filtering
2019-05-12 14:07:30,174 Converting to float64
2019-05-12 14:07:30,186 Resampling data
2019-05-12 14:07:31,039 Highpass Filtering
2019-05-12 14:07:31,161 Remove Padding
2019-05-12 14:07:31,162 Reading Frames
2019-05-12 14:07:40,453 Highp

In the next tutorial, we will take a look at an already completed result file.